# ETL Case study for Spar Nord Bank using sqoop, pyspark, S3 and redshift

In [1]:
#Import the necessary packages

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
import pyspark.sql.functions as F

VBox()

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Creating the spark session

spark = SparkSession.builder.appName("BANK ATM ETL CASE STUDY").getOrCreate()
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Creating the schema for reading the data from the input csv file

FileSchema = StructType([StructField('year', IntegerType(), nullable = True),
                        StructField('month', StringType(), True),
                        StructField('day', IntegerType(), True),
                        StructField('weekday', StringType(), True),
                        StructField('hour', IntegerType(), True),
                        StructField('atm_status', StringType(), True),
                        StructField('atm_id', StringType(), True),
                        StructField('atm_manufacturer', StringType(), True),
                        StructField('atm_location', StringType(), True),
                        StructField('atm_streetname', StringType(), True),
                        StructField('atm_street_number', IntegerType(), True),
                        StructField('atm_zipcode', IntegerType(), True),
                        StructField('atm_lat', DoubleType(), True),
                        StructField('atm_lon', DoubleType(), True),
                        StructField('currency', StringType(), True),
                        StructField('card_type', StringType(), True),
                        StructField('transaction_amount', IntegerType(), True),
                        StructField('service', StringType(), True),
                        StructField('message_code', StringType(), True),
                        StructField('message_text', StringType(), True),
                        StructField('weather_lat', DoubleType(), True),
                        StructField('weather_lon', DoubleType(), True),
                        StructField('weather_city_id', IntegerType(), True),
                        StructField('weather_city_name', StringType(), True),
                        StructField('temp', DoubleType(), True),
                        StructField('pressure', IntegerType(), True),
                        StructField('humidity', IntegerType(), True),
                        StructField('wind_speed', IntegerType(), True),
                        StructField('wind_deg', IntegerType(), True),
                        StructField('rain_3h', DoubleType(), True),
                        StructField('clouds_all', IntegerType(), True),
                        StructField('weather_id', IntegerType(), True),
                        StructField('weather_main', StringType(), True),
                        StructField('weather_description', StringType(), True)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Reading the data from the csv file using the schema created in above step. 
# As header is absent in data file, we have set header option to false

df = spark.read.csv('/user/hadoop/bank_atm_etl_case_study/part-m-00000', header = False, schema = FileSchema)
df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[year: int, month: string, day: int, weekday: string, hour: int, atm_status: string, atm_id: string, atm_manufacturer: string, atm_location: string, atm_streetname: string, atm_street_number: int, atm_zipcode: int, atm_lat: double, atm_lon: double, currency: string, card_type: string, transaction_amount: int, service: string, message_code: string, message_text: string, weather_lat: double, weather_lon: double, weather_city_id: int, weather_city_name: string, temp: double, pressure: int, humidity: int, wind_speed: int, wind_deg: int, rain_3h: double, clouds_all: int, weather_id: int, weather_main: string, weather_description: string]

In [5]:
# Dispalying 5 records from the dataframe which point to bank csv file
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency| card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-

In [6]:
# Checking the number of records in the input file. 
# This count is same as the count of records read from the source file via sqoop
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [7]:
# Show the schema of the dataframe

df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: st

In [8]:
# Display the columns present in the dataframe/input data file

df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'atm_id', 'atm_manufacturer', 'atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description']

## Creation of location dimension data frame

In [9]:
# Selecting the required columns from the dataframe for location dimention
# Also applying distinct functon to remove duplicate values

df_dim_location = df.select('atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Adding the primary key location_id using the row_number function.

df_dim_location = df_dim_location.select('atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon',F.row_number().over(Window.partitionBy().orderBy(df_dim_location['atm_location'])).alias("location_id"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Selecting the columns as per the location dimension table

dim_location = df_dim_location.selectExpr('location_id', 'atm_location as location', 'atm_streetname as streetname' , 'atm_street_number as street_number' , 'atm_zipcode as zipcode' , 'atm_lat as lat' , 'atm_lon as lon')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Showing the column names  of the location dimension dataframe.

dim_location.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['location_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon']

In [13]:
# Counting the number of records in the location dimension dataframe
# This count matches with the count present in the validation document shared with us

dim_location.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [14]:
# Showing all the 109 records of the location dimension dataframe
dim_location.show(109)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+--------------------+-------------+-------+------+------+
|location_id|            location|          streetname|street_number|zipcode|   lat|   lon|
+-----------+--------------------+--------------------+-------------+-------+------+------+
|          1|             Aabybro|        ÃƒËœstergade|            6|   9440|57.162|  9.73|
|          2|      Aalborg Hallen|        Europa Plads|            4|   9000|57.044| 9.913|
|          3|Aalborg Storcente...|            Hobrovej|          452|   9200|57.005| 9.876|
|          4|Aalborg Storcente...|            Hobrovej|          452|   9200|57.005| 9.876|
|          5|         Aalborg Syd|            Hobrovej|          440|   9200|57.005| 9.881|
|          6|           AalbÃƒÂ¦k|          Centralvej|            5|   9982|57.593|10.412|
|          7|              Aarhus|      SÃƒÂ¸nder Alle|           11|   8000|56.153|10.206|
|          8|              Aarhus|          Ceres Byen|           75|   8000|56.

## Creation of atm dimension data frame

In [15]:
# Selecting the required columns from the dataframe for atm dimension
# Also applying distinct functon to remove duplicate values

df_dim_atm = df.selectExpr('atm_id as atm_number', 'atm_manufacturer', 'atm_location as location', 'atm_streetname as streetname', 'atm_lat as lat', 'atm_lon as lon').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Left Join of atm dataframe with location datarame on columns lat, lon, location and streetname to get the location_id from location 

df_dim_atm = df_dim_atm.join(dim_location, how = "left", on = ['lat', 'lon', 'location','streetname'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Count the number of records in atm dimension
# This count is same as the one present in validation document

df_dim_atm.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [18]:
# Select only the columns that are required for atm dimension

df_dim_atm = df_dim_atm.select('atm_number', 'atm_manufacturer', 'location_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Adding the primary key atm_id using the row_number function.

df_dim_atm = df_dim_atm.select('atm_number', 'atm_manufacturer', 'location_id', F.row_number().over(Window.partitionBy().orderBy(df_dim_atm['atm_number'])).alias("atm_id"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Select the columns as per the columns present in the atm dimension table
dim_atm = df_dim_atm.selectExpr('atm_id', 'atm_number', 'atm_manufacturer', 'location_id as atm_location_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Show the columns present in atm dimension
df_dim_atm.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_number', 'atm_manufacturer', 'location_id', 'atm_id']

In [22]:
# Count the number of records in atm dimension. 
# This is done again ater joining with location dimension to make sure it matches with the counts before the join
# This count is same as the one present in validation doument

df_dim_atm.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [23]:
# Show all the 113 records present in atm dimension

dim_atm.show(113)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------------+---------------+
|atm_id|atm_number|atm_manufacturer|atm_location_id|
+------+----------+----------------+---------------+
|     1|         1|             NCR|             75|
|     2|        10|             NCR|             76|
|     3|       100|             NCR|             56|
|     4|       101|             NCR|             17|
|     5|       102|             NCR|              3|
|     6|       103| Diebold Nixdorf|            103|
|     7|       104|             NCR|             58|
|     8|       105| Diebold Nixdorf|             76|
|     9|       106|             NCR|             55|
|    10|       107| Diebold Nixdorf|             62|
|    11|       108|             NCR|             47|
|    12|       109| Diebold Nixdorf|              5|
|    13|        11|             NCR|             80|
|    14|       110| Diebold Nixdorf|             41|
|    15|       111| Diebold Nixdorf|              8|
|    16|       112| Diebold Nixdorf|          

## Creation of card type dimension data frame

In [24]:
# Selecting the required columns from the dataframe for card type dimention
# Also applying distinct functon to remove duplicate values

df_dim_card_type = df.select('card_type').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Counting the number of records in the card type dimension dataframe
# This count matches with the count present in the validation document shared with us

df_dim_card_type.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [26]:
# Adding the primary key card_type_id using the row_number function.

dim_card_type = df_dim_card_type.select(F.row_number().over(Window.partitionBy().orderBy(df_dim_card_type['card_type'])).alias("card_type_id"), 'card_type')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Showing all the 12 records from card type dimension data frame

dim_card_type.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|card_type_id|           card_type|
+------------+--------------------+
|           1|              CIRRUS|
|           2|             Dankort|
|           3|     Dankort - on-us|
|           4|         HÃƒÂ¦vekort|
|           5| HÃƒÂ¦vekort - on-us|
|           6|             Maestro|
|           7|          MasterCard|
|           8|  Mastercard - on-us|
|           9|                VISA|
|          10|        Visa Dankort|
|          11|Visa Dankort - on-us|
|          12|            VisaPlus|
+------------+--------------------+

## Creation of date dimension data frame

In [28]:
# Selecting the required columns from the dataframe for date dimention
# Also applying distinct functon to remove duplicate values

df_dim_date = df.select('year', 'month', 'day', 'hour', 'weekday').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Showing the sample records

df_dim_date.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+---+----+---------+
|year| month|day|hour|  weekday|
+----+------+---+----+---------+
|2017|August|  1|  11|  Tuesday|
|2017|August|  3|  21| Thursday|
|2017|  July| 29|  17| Saturday|
|2017|August|  3|  15| Thursday|
|2017|August| 13|   2|   Sunday|
|2017|  July|  6|  14| Thursday|
|2017|  July| 13|  16| Thursday|
|2017|  July| 30|  13|   Sunday|
|2017|August|  3|   5| Thursday|
|2017|August| 12|  22| Saturday|
|2017|  July| 14|   2|   Friday|
|2017|August|  2|  22|Wednesday|
|2017|August| 10|  18| Thursday|
|2017|  July|  3|  10|   Monday|
|2017|  July| 27|  18| Thursday|
|2017|  July| 11|  18|  Tuesday|
|2017|  July| 25|  11|  Tuesday|
|2017|  July| 11|   2|  Tuesday|
|2017|  July| 27|  15| Thursday|
|2017|  July| 23|  17|   Sunday|
+----+------+---+----+---------+
only showing top 20 rows

In [30]:
# Code to create full_date_time column in the date format 'yyyy-MMM-dd HH:mm:ss' using the existing columns
# year, month, day and hour 

df_dim_date = df_dim_date.withColumn('date_part', concat_ws('-', df_dim_date.year, df_dim_date.month, df_dim_date.day))
df_dim_date = df_dim_date.withColumn('time_part', concat_ws(':', df_dim_date.hour, lit('00'), lit('00')))
df_dim_date = df_dim_date.withColumn('full_date_time', concat_ws(' ', df_dim_date.date_part, df_dim_date.time_part))
pattern = 'yyyy-MMM-dd HH:mm:ss'
df_dim_date = df_dim_date.withColumn('full_date_time', unix_timestamp(df_dim_date.full_date_time, pattern).cast('timestamp'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# creating the primary key date_id using the row_number function
dim_date = df_dim_date.select(F.row_number().over(Window.partitionBy().orderBy(df_dim_date['full_date_time'])).alias("date_id"),'full_date_time', 'year', 'month', 'day', 'hour', 'weekday')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# Showing the sample records to check if date_id (PK) and full_date_time are populated correctly or not

dim_date.show(20) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+-------+---+----+-------+
|date_id|     full_date_time|year|  month|day|hour|weekday|
+-------+-------------------+----+-------+---+----+-------+
|      1|2017-01-01 00:00:00|2017|January|  1|   0| Sunday|
|      2|2017-01-01 01:00:00|2017|January|  1|   1| Sunday|
|      3|2017-01-01 02:00:00|2017|January|  1|   2| Sunday|
|      4|2017-01-01 03:00:00|2017|January|  1|   3| Sunday|
|      5|2017-01-01 04:00:00|2017|January|  1|   4| Sunday|
|      6|2017-01-01 05:00:00|2017|January|  1|   5| Sunday|
|      7|2017-01-01 06:00:00|2017|January|  1|   6| Sunday|
|      8|2017-01-01 07:00:00|2017|January|  1|   7| Sunday|
|      9|2017-01-01 08:00:00|2017|January|  1|   8| Sunday|
|     10|2017-01-01 09:00:00|2017|January|  1|   9| Sunday|
|     11|2017-01-01 10:00:00|2017|January|  1|  10| Sunday|
|     12|2017-01-01 11:00:00|2017|January|  1|  11| Sunday|
|     13|2017-01-01 12:00:00|2017|January|  1|  12| Sunday|
|     14|2017-01-01 13:00:00|2017|Januar

In [33]:
# Counting the number of records in the date dimension dataframe
# This count matches with the count present in the validation document shared with us

dim_date.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

## Creation of ATM transaction fact table dataframe

In [34]:
# Selecting all the data from the dataframe which was created from input data file

df_fact_atm_trans = df.select('*')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Counting the number of records in the FACT_ATM_TRANS dimension dataframe
# This count matches with the count present in the validation document shared with us

df_fact_atm_trans.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [36]:
# Renaming the transaction dataframe columns to match with the columns of location dimension dataframe, so that we can join

df_fact_atm_trans = df_fact_atm_trans.withColumnRenamed('atm_location','location').withColumnRenamed('atm_streetname','streetname').withColumnRenamed('atm_street_number','street_number').withColumnRenamed('atm_zipcode','zipcode').withColumnRenamed('atm_lat','lat').withColumnRenamed('atm_lon','lon')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Creation of foreign keys by joining the transaction datarame with location, atm, date and card type 

In [37]:
# Left join transaction dataframe with location dimension dataframe to get the location_id which is FK in transaction table.
df_fact_atm_trans = df_fact_atm_trans.join(dim_location, on = ['location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# Again count the number of records after the left join. This count is same before the join

df_fact_atm_trans.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [39]:
# Renaming the transaction dataframe columns to match with the columns of atm dimension dataframe, so that we can join

df_fact_atm_trans = df_fact_atm_trans.withColumnRenamed('atm_id', 'atm_number').withColumnRenamed('location_id', 'atm_location_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
# Left join transaction dataframe with atm dimension dataframe to get the atm_id which is FK in transaction table.

df_fact_atm_trans = df_fact_atm_trans.join(dim_atm, on = ['atm_number', 'atm_manufacturer', 'atm_location_id'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# Renaming atm_location_id as weather_loc_id which is present in transactions fact table

df_fact_atm_trans = df_fact_atm_trans.withColumnRenamed('atm_location_id', 'weather_loc_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# Again count the number of records after the left join. This count is same before the join

df_fact_atm_trans.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [43]:
# Left join transaction dataframe with date dimension dataframe to get the date_id which is FK in transaction table.

df_fact_atm_trans = df_fact_atm_trans.join(dim_date, on = ['year', 'month', 'day', 'hour', 'weekday'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# Again count the number of records after the left join. This count is same before the join

df_fact_atm_trans.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [45]:
# Left join transaction dataframe with card_type dimension dataframe to get the card_type_id which is FK in transaction table.

df_fact_atm_trans = df_fact_atm_trans.join(dim_card_type, on = ['card_type'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# Again count the number of records after the left join. 
# This count is same before the join and matching with the number of transactions present in the validation doc.

df_fact_atm_trans.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [47]:
# Finally selecting the columns required by transactions fact table and also adding the primary key trans_id using row_number function

fact_atm_trans = df_fact_atm_trans.select(F.row_number().over(Window.partitionBy().orderBy(df_fact_atm_trans['full_date_time'])).alias('trans_id'), 'atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 
'atm_status', 'currency', 'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h', 
'clouds_all', 'weather_id', 'weather_main', 'weather_description')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
# Showing sample transactions records

df_fact_atm_trans.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------+---+----+-------+----------+----------------+--------------+-----------+-----------+-------------+-------+------+-----+----------+--------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+------+-------+-------------------+------------+
|           card_type|year|  month|day|hour|weekday|atm_number|atm_manufacturer|weather_loc_id|   location| streetname|street_number|zipcode|   lat|  lon|atm_status|currency|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|atm_id|date_id|     full_date_time|card_type_id|
+--------------------+----+-------+---+----+-------+----------+----------------+--------------+-----------+-

## Write the dimension and fact data to S3 bucket

In [49]:
# Write the dimension and fact data to S3 bucket aas-etl-case-study which was created for our etl case study team

# For date dimension write into S3, we have used timestampFormat option to remove the TimeZone which was getting 
# added when writng into S3 bucket. Ex: column had T and Z chars as present in the value 2017-01-01T00:00:00.000Z 

dim_location.write.csv('s3://aas-etl-case-study/dim_location/')
dim_atm.write.csv('s3://aas-etl-case-study/dim_atm/')
dim_card_type.write.csv('s3://aas-etl-case-study/dim_card_type/')
dim_date.write.option("timestampFormat", "yyyy-MM-dd HH:mm:ss").csv('s3://aas-etl-case-study/dim_date/')
fact_atm_trans.write.csv('s3://aas-etl-case-study/fact_atm_trans/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…